In [10]:
%load_ext autoreload
%autoreload 2

import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, Mask, PinMap, Metadata, Collection)
from reflexgenerator.io import load

from pathlib import Path
import pandas as pd
from reflexgenerator.generator.xref import UidReference


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of reflexgenerator.sources failed: Traceback (most recent call last):
  File "c:\Users\neurogears\anaconda3\envs\GitHarpCrawler\lib\site-packages\IPython\extensions\autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\neurogears\anaconda3\envs\GitHarpCrawler\lib\site-packages\IPython\extensions\autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\neurogears\anaconda3\envs\GitHarpCrawler\lib\site-packages\IPython\extensions\autoreload.py", line 381, in update_generic
    update(a, b)
  File "c:\Users\neurogears\anaconda3\envs\GitHarpCrawler\lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    update_instances(old, new)
  File "c:\Users\neurogears\anaconda3\envs\GitHarpCrawler\lib\site-packages\IPython\extensions\autoreload.py", line 307, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'Metadata' object layout d

In [11]:
schema = r"schema\device.json"
fw_target = r"schema\example.yml"


In [12]:
jsondata = load(fw_target)


In [13]:
metadata = Metadata(
    **{"device": jsondata["device"],
    "whoAmI": jsondata["whoAmI"],
    "architecture": jsondata["architecture"],
    "firmwareVersion": jsondata["firmwareVersion"],
    "hardwareTargets": jsondata["hardwareTargets"]})
print(metadata)

Metadata(device='ExampleDevice', whoAmI=0, firmwareVersion='0.1', hardwareTargets='0.1', architecture='atmega', uid=ref-Device-Metadata-ExampleDevice_0)


In [14]:
# build masks
if "bitMasks" in jsondata:
    bitMasks = Collection([Mask.from_json(mask) for mask in jsondata["bitMasks"].items() if mask is not None])
    bitMasks_df = pd.DataFrame([mask.to_dict() for mask in bitMasks])
    bitMasks_df.head(5)
else:
    bitMasks = None
    bitMasks_df = None

if "groupMask" in jsondata:
    groupMasks = Collection([Mask.from_json(mask) for mask in jsondata["groupMask"].items() if mask is not None])
    groupMasks_df = pd.DataFrame([mask.to_dict() for mask in groupMasks])
    groupMasks_df.head(5)
else:
    groupMasks = None
    groupMasks_df = None

In [15]:
if "registers" in jsondata:
    regs = Collection([Register.from_json(reg) for reg in jsondata["registers"].items() if reg is not None])
    register_df = pd.DataFrame([reg.to_dict() for reg in regs])
    register_df.head(5)


In [16]:
register_df

,name,address,payloadType,payloadLength,registerType,payloadSpec,maskType,description,converter,visibility,group,uid
0,Cam0Event,32,PayloadType.U8,1,RegisterType.Event,None,None,None,None,VisibilityType.Public,None,ref-Device-Register-Cam0Event
1,Cam0TriggerFrequency,33,PayloadType.U16,1,RegisterType.Command,None,"[Mask(name='DO', description='Bitmask represen...",Sets the trigger frequency for camera 0 betwee...,None,VisibilityType.Public,None,ref-Device-Register-Cam0TriggerFrequency
2,Cam0TriggerDuration,34,PayloadType.U16,1,RegisterType.Command,None,None,Sets the duration of the trigger pulse (minimu...,None,VisibilityType.Public,None,ref-Device-Register-Cam0TriggerDuration
3,StartAndStop,35,PayloadType.U8,1,RegisterType.Command,None,None,Starts or stops the camera immediately.,None,VisibilityType.Public,None,ref-Device-Register-StartAndStop
4,InState,36,PayloadType.U8,1,RegisterType.Event,None,None,Contains the state of the input ports.,None,VisibilityType.Public,None,ref-Device-Register-InState
5,Valve0Pulse,37,PayloadType.U8,1,RegisterType.Command,None,None,Configures the valve 0 open time in milliseconds.,None,VisibilityType.Public,None,ref-Device-Register-Valve0Pulse
6,OutSet,38,PayloadType.U8,1,RegisterType.Command,None,None,Bitmask to set the available outputs.,None,VisibilityType.Public,None,ref-Device-Register-OutSet
7,OutClear,39,PayloadType.U8,1,RegisterType.Command,None,None,Bitmask to clear the available outputs.,None,VisibilityType.Public,None,ref-Device-Register-OutClear
8,OutToggle,40,PayloadType.U8,1,RegisterType.Command,None,None,Bitmask to toggle the available outputs.,None,VisibilityType.Public,None,ref-Device-Register-OutToggle
9,OutWrite,41,PayloadType.U8,1,RegisterType.Command,None,"[Mask(name='DO', description='Bitmask represen...",Bitmask to write the available outputs.,None,VisibilityType.Public,None,ref-Device-Register-OutWrite


In [17]:
if "ios" in jsondata:
    pinMapping = Collection([PinMap.from_json(pinmap) for pinmap in jsondata["ios"].items() if pinmap is not None])
    pinMapping_df = pd.DataFrame([pinmap.to_dict() for pinmap in pinMapping])
    pinMapping_df.head(5)
pinMapping_df

,name,port,pin,direction,useInput,pull,sense,interruptPriority,interruptNumber,out,outDefault,outInvert,description,uid
0,DO3,PORTC,0,output,None,None,None,None,None,None,None,None,DO0,ref-Device-PinMap-DO3
1,DO2,PORTB,2,output,None,None,None,None,None,None,None,None,DO0,ref-Device-PinMap-DO2
